In [5]:
import numpy as np
import pandas as pd
import json
import gzip
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


# 'C:\\Users\\emovi\\Desktop\\VibeCaster\\VibeCaster\\data\\Software_5.json'
#C:\Users\emovi\Desktop\VibeCaster\VibeCaster\data\Software_5.json(1).gz

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GoekcekE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
## load the datset into DataFrame as described on the website: https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/#subsets
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
# small dataset, ca. 77 000 Entries
# df = getDF('C:\\Users\\emovi\\Desktop\\VibeCaster\\VibeCaster\\data\\Industrial_and_Scientific_5.json.gz')
# way bigger dataset ca. 500 000 Entries
# df = getDF('C:\\Users\\emovi\\Desktop\\VibeCaster\\VibeCaster\\data\\Video_Games_5.json.gz')
df = getDF('C:\\Users\\GoekcekE\OneDrive - Mitsubishi Electric Europe B.V\\Desktop\\VibeCasterDev\\VibeCaster\\data\Video_Games_5.json.gz')

In [7]:
 # summary statistics
print(df.shape) # 
print(df.info())
print(df.describe())

(497577, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 497577 entries, 0 to 497576
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         497577 non-null  float64
 1   verified        497577 non-null  bool   
 2   reviewTime      497577 non-null  object 
 3   reviewerID      497577 non-null  object 
 4   asin            497577 non-null  object 
 5   reviewerName    497501 non-null  object 
 6   reviewText      497419 non-null  object 
 7   summary         497468 non-null  object 
 8   unixReviewTime  497577 non-null  int64  
 9   vote            107793 non-null  object 
 10  style           289237 non-null  object 
 11  image           3634 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 46.0+ MB
None
             overall  unixReviewTime
count  497577.000000    4.975770e+05
mean        4.220456    1.367848e+09
std         1.185424    1.224113e+08
min         

In [8]:
print(df.isna().sum())

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          76
reviewText           158
summary              109
unixReviewTime         0
vote              389784
style             208340
image             493943
dtype: int64


In [9]:
# delete entrys with missing reviewText: 
df.dropna(subset=['reviewText', 'overall'], inplace=True)

In [10]:
# only keep overall and reviewText
all_columns = df.columns.tolist()

columns_to_keep = ['overall', 'reviewText']

columns_to_drop = [col for col in all_columns if col not in columns_to_keep]

df.drop(columns=columns_to_drop, inplace=True)

In [11]:
print(df.describe())

             overall
count  497419.000000
mean        4.220297
std         1.185491
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000


In [12]:
# create new col sentiment to train the model on
df['sentiment'] = df['overall'].apply(lambda x: 'positive' if x > 3 else ('neutral' if x == 3 else 'negative'))
# Count the number of each sentiment
print(df['sentiment'].value_counts())


sentiment
positive    393267
negative     55012
neutral      49140
Name: count, dtype: int64


In [13]:
## dataset is heavily skewed on the positive side of things so might need to use cross-validation to account for it
# we only need reviewText and Sentiment to start training our model so lets drop the 'overall' column
df.drop(columns="overall", inplace=True)


In [14]:
# Using the BERT tokenizer to preprocess the text.
# The function `tokenize_reviews` encodes the text into token IDs, adds special tokens ([CLS], [SEP]), and pads the sequence to a maximum length of 512 tokens.
# It also generates an attention mask to differentiate real tokens from [PAD] tokens.
# The resulting 'data' column in the DataFrame will contain the token IDs and attention masks for each review.
 
from transformers import BertTokenizer
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_reviews(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
)

# Apply tokenization to the reviews
df['data'] = df['reviewText'].apply(tokenize_reviews)

c:\Users\GoekcekE\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\GoekcekE\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GoekcekE\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order 

In [15]:
# splitting data into test and trainingsdata to measure performance later
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
max_len = max(item['input_ids'].shape[1] for item in train['data'])

def pad_tensor(dict_item):
    input_ids = dict_item['input_ids']
    attention_mask = dict_item['attention_mask']
    
    padded_input_ids = torch.cat([input_ids, torch.zeros(1, max_len - input_ids.shape[1])], dim=1)
    padded_attention_mask = torch.cat([attention_mask, torch.zeros(1, max_len - attention_mask.shape[1])], dim=1)
    
    dict_item['input_ids'] = padded_input_ids
    dict_item['attention_mask'] = padded_attention_mask
    
    return dict_item

train['data'] = train['data'].apply(pad_tensor)
# If max_len for test data should be the same as that for train data, you can skip this step.
# Otherwise, uncomment and run the line below:
# max_len = max(item['input_ids'].shape[1] for item in test['data'])

# Apply padding to test data
test['data'] = test['data'].apply(pad_tensor)


In [18]:
# Creating PyTorch tensors for training and testing data.
# Concatenating input_ids and attention_masks for each dataset to form tensors.
# Also converting sentiment labels to integer form and forming corresponding tensors.
# This prepares the data in the format required for BERT model training and evaluation.

import torch

train_data = torch.cat([item['input_ids'] for item in train['data']], dim=0)
train_attention_masks = torch.cat([item['attention_mask'] for item in train['data']], dim=0)
train_labels = torch.tensor(train['sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0}).values)

test_data = torch.cat([item['input_ids'] for item in test['data']], dim=0)
test_attention_masks = torch.cat([item['attention_mask'] for item in test['data']], dim=0)
test_labels = torch.tensor(test['sentiment'].map({'positive': 2, 'neutral': 1, 'negative': 0}).values)


In [19]:
# create mini-batches of the data
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Combine the training inputs into a TensorDataset
train_dataset = TensorDataset(train_data, train_attention_masks, train_labels)

# Create a DataLoader
train_dataloader = DataLoader(
    train_dataset,  # The training samples.
    sampler=RandomSampler(train_dataset), # Select batches randomly
    batch_size=32 # Trains with this batch size.
)

In [24]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=3, # The number of output labels--3 for binary classification.
    output_attentions=False, # Whether the model returns attentions weights.
    output_hidden_states=False, # Whether the model returns all hidden-states.
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
device = torch.device("cpu")

# Initialize AdamW optimizer
optimizer = AdamW(model.parameters(),
                  lr=2e-5,  # Learning rate
                  eps=1e-8  # Epsilon value to prevent division by zero
                 )
from tqdm import tqdm  # For progress bar
import torch.nn.functional as F  # For the softmax function

# Number of training epochs
epochs = 4

# Loop over each epoch
for epoch in tqdm(range(epochs), desc="Epoch"):

    # Put the model into training mode
    model.train()

    # Initialize loss for this epoch
    total_loss = 0

    # Loop over each batch of training data
    for step, batch in enumerate(train_dataloader):

        # Unpack batch from dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Move batch tensors to the same device as the model
        b_input_ids = b_input_ids.long().to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)

        # Zero out any previously calculated gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # Get the loss from the outputs
        loss = outputs[0]

        # Add loss to the total loss
        total_loss += loss.item()

        # Perform a backward pass to calculate gradients
        loss.backward()

        # Update parameters
        optimizer.step()

    # Calculate the average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)
    
    print(f"Average training loss: {avg_train_loss}")



c:\Users\GoekcekE\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

: 